In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from mgwr.gwr import GWR
import statsmodels.api as sm
from mgwr.sel_bw import Sel_BW
import libpysal as lps
from esda.moran import Moran
import statsmodels.formula.api as smf

### Prep Dataset

In [22]:
df = gpd.read_file("../data/final_dataset_mest.geojson")

df['crime_density'] = df['crime_total'] / df['area_km2']
df['crime_attractor_density'] = (df['poi_pub'] + df['poi_nightclub'] + df['poi_bar']) / df['area_km2']
df['crime_generator_density'] = (df['poi_cafe'] + df['poi_museum'] + df['poi_attraction']) / df['area_km2']
df['tourist_density'] = df['nights_non_residents'] / df['area_km2']
df['foreigner_density'] = df['foreigners'] / df['area_km2']
df['university_density'] = df['people_university'] / df['area_km2']
df['airbnb_economy_density'] = df['airbnb_economy'] / df['area_km2']
df['crime_Burglary_density'] = df['crime_Burglary'] / df['area_km2']
df['crime_Disorder_density'] = df['crime_Disorder'] / df['area_km2']
df['crime_Theft_density'] = df['crime_Theft'] / df['area_km2']
df['crime_Violent_density'] = df['crime_Violent'] / df['area_km2']
df['crime_Other_density'] = df['crime_Other'] / df['area_km2']

df = df.fillna(0.0)

df_clean = df.drop([2, 8, 21, 46, 56])

### GLM for comparison

- since VIF and LASSO both dropped airbnb_luxury_density, we will only fit the economy model

Final VIF Table:
                   feature       VIF
1  airbnb_economy_density  4.245560
2         tourist_density  3.443399
0          dist_center_km  1.482178
3            unemployment  1.118987


Lasso Coefficients:
 mhd_density                1.556682
crime_generator_density    1.326616
foreigner_density          1.260337
tourist_density            0.000000
unemployment              -0.113861
dist_center_km            -0.245982
airbnb_economy_density    -0.338079
university_density        -0.816004
crime_attractor_density   -1.845016

In [23]:
features_economy_vif = ['airbnb_economy_density', 
                        'tourist_density', 
                        'dist_center_km', 
                        'unemployment']

features_economy_lasso = ['airbnb_economy_density',
                          'mhd_density', 
                          'crime_generator_density',
                          'foreigner_density',
                          'unemployment',
                          'dist_center_km', 
                          'university_density', 
                          'crime_attractor_density']

In [25]:
df_economy_vif = df_clean[features_economy_vif + ['geometry'] + ['crime_density']].copy()

formula_vif = ('np.log1p(crime_density) ~ dist_center_km + I(dist_center_km**2) + '
               'np.log1p(airbnb_economy_density) + np.log1p(tourist_density) + '
               'unemployment')

glm_vif = smf.glm(
    formula=formula_vif,
    data=df_economy_vif,
    family=sm.families.Gaussian()
).fit()

print(glm_vif.summary())

                    Generalized Linear Model Regression Results                    
Dep. Variable:     np.log1p(crime_density)   No. Observations:                   52
Model:                                 GLM   Df Residuals:                       46
Model Family:                     Gaussian   Df Model:                            5
Link Function:                    Identity   Scale:                         0.65808
Method:                               IRLS   Log-Likelihood:                -59.718
Date:                     Wed, 31 Dec 2025   Deviance:                       30.272
Time:                             15:10:00   Pearson chi2:                     30.3
No. Iterations:                          3   Pseudo R-squ. (CS):             0.8976
Covariance Type:                 nonrobust                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------

In [26]:
df_economy_lasso = df_clean[features_economy_lasso + ['geometry'] + ['crime_density']].copy()

formula_lasso = ('np.log1p(crime_density) ~ '
                 'dist_center_km + I(dist_center_km**2) + '
                 'np.log1p(airbnb_economy_density) + '
                 'np.log1p(mhd_density) + '
                 'np.log1p(crime_generator_density) + '
                 'np.log1p(foreigner_density) + '
                 'unemployment + '
                 'np.log1p(university_density) + '
                 'np.log1p(crime_attractor_density)')

glm_lasso = smf.glm(
    formula=formula_lasso,
    data=df_economy_lasso,
    family=sm.families.Gaussian()
).fit()

print(glm_lasso.summary())

                    Generalized Linear Model Regression Results                    
Dep. Variable:     np.log1p(crime_density)   No. Observations:                   52
Model:                                 GLM   Df Residuals:                       42
Model Family:                     Gaussian   Df Model:                            9
Link Function:                    Identity   Scale:                         0.45319
Method:                               IRLS   Log-Likelihood:                -47.655
Date:                     Wed, 31 Dec 2025   Deviance:                       19.034
Time:                             15:10:00   Pearson chi2:                     19.0
No. Iterations:                          3   Pseudo R-squ. (CS):             0.9776
Covariance Type:                 nonrobust                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------

### Test for spatial autocorrelation

In [27]:
def spatial_autocorrelation_test(df, residuals):
    w = lps.weights.Queen.from_dataframe(df)
    w.transform = 'r'

    moran_resid = Moran(residuals, w)
    print(f"Moran's I (GLM residuals): {moran_resid.I:.4f}")
    print(f"p-value: {moran_resid.p_sim:.4f}")

    if moran_resid.p_sim < 0.05:
        print("spatial autocorrelation detected → GWR")
    else:
        print("no spatial autocorrelation → OLS is enough")


spatial_autocorrelation_test(df_economy_vif, glm_vif.resid_response)

spatial_autocorrelation_test(df_economy_lasso, glm_lasso.resid_response)


Moran's I (GLM residuals): 0.1114
p-value: 0.0980
no spatial autocorrelation → OLS is enough
Moran's I (GLM residuals): 0.0780
p-value: 0.1360
no spatial autocorrelation → OLS is enough


C:\Users\Kaja\AppData\Local\Temp\ipykernel_46632\2874209154.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = lps.weights.Queen.from_dataframe(df)


- makes sense since the most dense central (crime and airbnb) districts overrule the periphery
- we can still use GWR to see potential differences between districts

### GWR

#### VIF-chosen features

In [28]:
df_vif_gwr = df_economy_vif.copy()
df_vif_gwr['log_crime_density'] = np.log1p(df_economy_vif['crime_density'])
df_vif_gwr['log_airbnb_economy'] = np.log1p(df_economy_vif['airbnb_economy_density'])
df_vif_gwr['log_tourist_density'] = np.log1p(df_economy_vif['tourist_density'])
df_vif_gwr['dist_center_km2'] = df_economy_vif['dist_center_km'] ** 2

y_vif = df_vif_gwr['log_crime_density'].values.reshape(-1, 1)
X_vif = df_vif_gwr[[
    'log_airbnb_economy',
    'log_tourist_density',
    'dist_center_km',
    'dist_center_km2',
    'unemployment'
]].values

X_vif_with_intercept = np.hstack([np.ones((len(X_vif), 1)), X_vif])

coords_vif = np.array(list(zip(
    df_vif_gwr.geometry.centroid.x,
    df_vif_gwr.geometry.centroid.y
)))


In [29]:
y_arr = np.array(y_vif).reshape(-1, 1)
X_arr = np.array(X_vif_with_intercept)
coords_arr = np.array(coords_vif)

bw_vif = Sel_BW(coords_arr, 
                y_arr, 
                X_arr, 
                multi=False, 
                constant=False).search(bw_min=2, bw_max=len(y_arr))
gwr_vif = GWR(coords_vif, y_vif, X_vif_with_intercept, bw_vif).fit()
print(f"  Bandwidth: {bw_vif:.2f}")
print(f"  R-squared: {gwr_vif.R2:.4f}")
print(f"  AICc: {gwr_vif.aicc:.2f}")

#print(gwr_vif.params.shape)

df_vif_gwr['gwr_local_r2'] = gwr_vif.localR2
df_vif_gwr['gwr_intercept'] = gwr_vif.params[:, 0]
df_vif_gwr['gwr_airbnb_coef'] = gwr_vif.params[:, 1]
df_vif_gwr['gwr_tourist_coef'] = gwr_vif.params[:, 2]
df_vif_gwr['gwr_centerd_coef'] = gwr_vif.params[:, 3]
df_vif_gwr['gwr_centerd2_coef'] = gwr_vif.params[:, 4]
df_vif_gwr['gwr_unemployment_coef'] = gwr_vif.params[:, 5]

df_vif_gwr['gwr_airbnb_t'] = gwr_vif.tvalues[:, 1]
df_vif_gwr['gwr_tourist_t'] = gwr_vif.tvalues[:, 2]
df_vif_gwr['gwr_centerd_t'] = gwr_vif.tvalues[:, 3]
df_vif_gwr['gwr_centerd2_t'] = gwr_vif.tvalues[:, 4]
df_vif_gwr['gwr_unemployment_t'] = gwr_vif.tvalues[:, 5]

df_vif_gwr['gwr_airbnb_se'] = gwr_vif.std_res


  Bandwidth: 51.00
  R-squared: 0.7877
  AICc: 134.46


#### LASSO-chosen features

In [30]:
df_lasso_gwr = df_economy_lasso.copy()
df_lasso_gwr['log_crime_density'] = np.log1p(df_economy_lasso['crime_density'])
df_lasso_gwr['log_airbnb_economy'] = np.log1p(df_economy_lasso['airbnb_economy_density'])
df_lasso_gwr['log_mhd'] = np.log1p(df_economy_lasso['mhd_density'])
df_lasso_gwr['log_crime_gen'] = np.log1p(df_economy_lasso['crime_generator_density'])
df_lasso_gwr['log_foreigner'] = np.log1p(df_economy_lasso['foreigner_density'])
df_lasso_gwr['log_university'] = np.log1p(df_economy_lasso['university_density'])
df_lasso_gwr['log_crime_attr'] = np.log1p(df_economy_lasso['crime_attractor_density'])
df_lasso_gwr['dist_center_km2'] = df_economy_lasso['dist_center_km'] ** 2

y_lasso = df_lasso_gwr['log_crime_density'].values.reshape(-1, 1)
X_lasso = df_lasso_gwr[[
    'log_airbnb_economy',
    'log_mhd',
    'log_crime_gen',
    'log_foreigner',
    'unemployment',
    'dist_center_km',
    'dist_center_km2',
    'log_university',
    'log_crime_attr'
]].values

X_lasso_with_intercept = np.hstack([np.ones((len(X_lasso), 1)), X_lasso])

coords_lasso = np.array(list(zip(
    df_lasso_gwr.geometry.centroid.x,
    df_lasso_gwr.geometry.centroid.y
)))

In [31]:
y_arr = np.array(y_lasso).reshape(-1, 1)
X_arr = np.array(X_lasso_with_intercept)
coords_arr = np.array(coords_lasso)

bw_lasso = Sel_BW(coords_arr, 
                y_arr, 
                X_arr, 
                multi=False, 
                constant=False).search(bw_min=2, bw_max=len(y_arr))

gwr_lasso = GWR(coords_lasso, y_lasso, X_lasso_with_intercept, bw_lasso).fit()
print(f"  Bandwidth: {bw_lasso:.2f}")
print(f"  R²: {gwr_lasso.R2:.4f}")
print(f"  AICc: {gwr_lasso.aicc:.2f}")

df_lasso_gwr['gwr_local_r2'] = gwr_lasso.localR2
df_lasso_gwr['gwr_intercept'] = gwr_lasso.params[:, 0]

df_lasso_gwr['gwr_airbnb_coef'] = gwr_lasso.params[:, 1]
df_lasso_gwr['gwr_mhd_coef'] = gwr_lasso.params[:, 2]
df_lasso_gwr['gwr_crimegen_coef'] = gwr_lasso.params[:, 3]
df_lasso_gwr['gwr_foreigner_coef'] = gwr_lasso.params[:, 4]
df_lasso_gwr['gwr_unemployment_coef'] = gwr_lasso.params[:, 5]
df_lasso_gwr['gwr_centerd_coef'] = gwr_lasso.params[:, 6]
df_lasso_gwr['gwr_centerd2_coef'] = gwr_lasso.params[:, 7]
df_lasso_gwr['gwr_university_coef'] = gwr_lasso.params[:, 8]
df_lasso_gwr['gwr_crimeattr_coef'] = gwr_lasso.params[:, 9]

df_lasso_gwr['gwr_airbnb_t'] = gwr_lasso.tvalues[:, 1]
df_lasso_gwr['gwr_mhd_t'] = gwr_lasso.tvalues[:, 2]
df_lasso_gwr['gwr_crimegen_t'] = gwr_lasso.tvalues[:, 3]
df_lasso_gwr['gwr_foreigner_t'] = gwr_lasso.tvalues[:, 4]
df_lasso_gwr['gwr_unemployment_t'] = gwr_lasso.tvalues[:, 5]
df_lasso_gwr['gwr_centerd_t'] = gwr_lasso.tvalues[:, 6]
df_lasso_gwr['gwr_centerd2_t'] = gwr_lasso.tvalues[:, 7]
df_lasso_gwr['gwr_university_t'] = gwr_lasso.tvalues[:, 8]
df_lasso_gwr['gwr_crimeattr_t'] = gwr_lasso.tvalues[:, 9]

  Bandwidth: 51.00
  R²: 0.8811
  AICc: 132.32
